# Introduction

### The Framework
The following notebook demonstrates an initial proof of concept for a Python 3.6 based motion analysis framework. The framework is inspired by the work proposed in *Camurri et al. (2016)*. Simply explained, this framework is divided into four levels of abstration. The first level revolves around acquisition and pre-processing physical signals (e.g. as captured by a sensor) or similar. The second level operates on the physical signals to extract low-level motion features, for example kinematics. The third level further operates on the low-level features to obtain "mid-level" features, for example in the form of motion descriptors, e.g. through the Laban effort descriptors. Finally, the fourth level attempts to make sense of the mid-level descriptors, and provide a label to these motion qualities in a manner which an external observer might deduce. 

### The present implementation
The framework proposed above does not require any specific implementation platform or approach to the various computations within each level of abstraction. The present implementation delegates the functionality required within each level to a subset of classes. 

*Most of the implementation for these classes are presented directly as source code within this notebook for the sake of the reader. In reality, they all reside within their own .py files and are collected in a dedicated Python-package  called pymoco, which will be available.*

The motion signal acquisition and subsequent computing has been derived from various other sources present in the litterature, and will be referred throughout this notebook.


#### Reference:
<sup>Camurri, Antonio, et al. "The dancer in the eye: towards a multi-layered computational framework of qualities in movement." *Proceedings of the 3rd International Symposium on Movement and Computing.* 2016.</sup>

---


# *Physical Signals* Acquisition

The acquistion of motion signals are not acquired in real-time. The current implementation supports parsing the Acclaim Format (ASF/AMC), which is a text based format for encoding motion captured movement signals. The format has two file types: ASF (Acclaim Skeleton File), which defines the markers (i.e. joints), the coordinate system (XYZ, ZXY, etc.), measurement (positions or angles) and degrees of freedom (dof) for each marker. Each motion capture session has just a single ASF file. Every subsequent sample is stored in the AMC (Acclaim Motion Capture) file, with the corresponding position or angle for each marker (see [Acclaim Skeleton File Definition V1.10](http://graphics.cs.cmu.edu/nsp/course/cs229/info/Acclaim_Skeleton_Format.html) or [CMU Description](https://research.cs.wisc.edu/graphics/Courses/cs-838-1999/Jeff/ASF-AMC.html) for more information). 

---

The code block below imports definitions for two classes responsible for acquiring the relevant motion capture data. The `AcclaimFileHandler` defines methods for parsing the Acclaim Format described above. To increase processing speed, the heavy-weight computations are implemented in the C-language and wrapped in Python through the Cython extension module. The `CMUScanner` class defines methods for fetching ASF/ACM files from the Carnegie Mellon Univeristy's (CMU) motion capture database. The CMU database has 300+ motion captured movements of a large variety (e.g. http://mocap.cs.cmu.edu/subjects.php).

In [1]:
# ------- "LOCAL/CUSTOM FILES" ---------- #
#
from acclaimhandler import AcclaimFileHandler
from cmuscanner import CMUScanner
#
# --------------------------------------- #

### Import additional computation libraries
Below are imported class definitions from additional 3rd party libraries. 

In [2]:
import inspect

import numpy as np
import pandas as pd

from scipy.signal import filtfilt
from scipy.spatial import ConvexHull

import matplotlib as mpl
import plotly.offline as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Interfaces for throwing run-time errors
Below is simply two class definitions that can be implemented in any class to throw run-time execptions.

In [3]:
class Error(Exception):
    pass


class InputError(Error):

    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

### Motion Data and Preprocesing
The `MotionExtractor` class below is an abstract representation of the first level described in the aforementioned framework (e.g. *Camurri et al. (2016)*). The class is responsible for parsing motion-capture data and, if desired, filter the signals. The parsing is achieved using the approach described above (*Physical Signal* Acquisition). The filtering is achieved through a 4-th order IIR-filter as described in *Skogstad & Jensenius (2013)*. The class implements a look-up table which holds the appropriate filter-coefficients for a desired (normalized) cut-off frequency. 

#### Reference:
<sup>Skogstad, S. A. V. D., Nymoen, K., Høvin, M. E., Holm, S., & Jensenius, A. R. (2013). Filtering Motion Capture Data for Real-Time Applications. NIME.</sup>


In [4]:
class MotionExtractor(object):
    
    CUTOFFS = ['0.1', '0.2', '0.3', '0.4', '0.5']
    
    def __init__(self):
        
        self.lpf_coeffs = {
            '0.1': [
                [0.1400982208, -0.0343775491, 0.0454003083, 0.0099732061, 0.0008485135],
                [1, -1.9185418203, 1.5929378702, -0.5939699187, 0.0814687111]
            ],
            '0.2': [
                [0.1526249789, 0.0333481282, 0.0777551903, 0.0667145281, 0.0138945068],
                [1, -1.7462227354, 1.7354077932, -0.8232679111, 0.1793463694]
            ],
            '0.3': [
                [0.1851439645, 0.1383283833, 0.1746892243, 0.1046627716, 0.0464383730],
                [1, -1.2982434912, 1.4634092217, -0.7106501488, 0.2028836637]
            ],
            '0.4': [
                [0.2680960849, 0.5174415712, 0.5839923942, 0.3748650443, 0.1199394960],
                [1, 0.0324610402, 0.7694515981, -0.0071430949, 0.0714586993]
            ],
            '0.5': [
                [0.3730569536, 0.8983119412, 0.9660856693, 0.5189611913, 0.1099005390],
                [1, 0.8053107424, 0.8110594452, 0.2371869724, 0.0849291749]
            ]
        }
        
        self.parse_params = {
            'path': None,
            'subject_number': 12,
            'trial_id': 'Trial4',
        }
        
        self.cmu_scanner = CMUScanner()
        self.afh = AcclaimFileHandler()
        
    
    def set_parse_params(self, path, **kwargs):
        self.parse_params['path'] = path
        if path: return
        
        for key, value in kwargs.items():
            if not key in self.parse_params:
                s = inspect.stack()[0].function
                raise InputError (
                    'An error occured while calling function \"%s\"'% s,
                    'Unknown parameter: \"%s\"'% key
                )
            
            self.parse_params[key] = value
            
    def parse_motion(self):
        path = self.parse_params['path']        
        if path: 
            return self.__parse_local(path)
        
        return self.__parse_remote()
        
    
    def __parse_local(self, path):
        pass
    
    def __parse_remote(self):
        subject_number = self.parse_params['subject_number']
        trial_id = self.parse_params['trial_id']
        
        subject_data = self.cmu_scanner.scan_cmu(subject_number)
        asf_url = subject_data['asf']
        t4_info = subject_data[trial_id]
        amc_url = t4_info['amc']

        framerate = t4_info['FrameRate']
        
        self.afh.manual_parse_asf(asf_url, 1)
        self.afh.manual_parse_amc(amc_url, 1)

        data, hier = self.afh.load_poses() 
        
        return (data, hier, framerate)
        
    def filter_motion(self, motion, cutoff='0.1'):
        if not cutoff in MotionExtractor.CUTOFFS:
            s = inspect.stack()[0].function
            raise InputError (
                'An error occured while calling function \"%s\"'% s,
                'Allowed normalized cutoffs are: 0.1, 0.2, 0.3, 0.4, 0.5'
            )
            
        coeffs = self.lpf_coeffs[cutoff]
        b, a = coeffs[0], coeffs[1]
        
        ndof = motion.shape[-1]
        out_motion = np.zeros_like(motion)
        for i in range(motion.shape[1]):
            signal = motion[:, i]
            y = self.__filter_1d_sig(b, a, signal, ndof)
            out_motion[:, i] = y
        
        return out_motion
        
    def __filter_1d_sig(self, b, a, signal, ndof):
        out_signals = []
        for dof in range(0, ndof):
            x = signal[:, dof]
            y = filtfilt(b, a, x)
            out_signals.append(y)

        return np.vstack((tuple(out_signals))).T
    
    
    def __del__(self):
        self.afh.manual_dealloc()

# Low-level Descriptors
The `MotionDescriptor` class below defines methods to compute low-level descriptors on the *"physical signals"* obtained via, e.g., the `MotionExtractor` class. Evidently, this class is the abstract representation of the second level of the framework in *Camurri et al. (2016)*. The class can compute descriptors such as the center of mass (CoM), quantity of motion (QoM), curvature (both 2D and 3D), and the three kinematic descriptors, velocity, acceleration and jerk. Computation formulas are derived from *Larboulette & Gibet (2015)*. The kinematic descriptors are estimated via central differences. 

In addition, the class reveals methods for estimating the convex hull, bounding sphere and bounding box sorrounding a set of markers provided as an argument to the function. 

#### Reference:
<sup>Caroline Larboulette and Sylvie Gibet. 2015. A review of computable expressive descriptors of human motion. In Proceedings of the 2nd International Workshop on Movement and Computing (MOCO ’15). Association for Computing Machinery, New York, NY, USA, 21–28. DOI:https://doi.org/10.1145/2790994.2790998</sup>


In [5]:
class MotionDescriptor(object):
    
    def __init__(self):
        self.wrap_func = self.__get_wrap_func()
        self.eps = 1e-10
        
    def norm(self, ndsig):
        return np.linalg.norm(ndsig, axis=-1)
    
    def center_of_mass(self, x, markers=[], weights=[]):
        if not np.any(markers):
            markers = np.arange(x.shape[1])
            
        if not np.any(weights):
            weights = np.ones(markers.shape) / len(markers)
        
        out = np.zeros_like(x[:, markers, :])
        for dof in range(x.shape[2]):
            out[:, :, dof] = np.multiply(x[:, markers, dof], weights)
        
        return np.sum(out, axis=1) / np.sum(weights)
        
    def quantity_of_motion(self, vel, weights=[]):
        marker_dim = vel.shape[-2]
        if len(weights) is not marker_dim:
            weights = np.ones(marker_dim) / marker_dim
            
        speed_k = self.norm(vel)
        return np.sum(np.multiply(speed_k, weights), axis=-1) / np.sum(weights)
    
    def velocity(self, x, dt=1):
        vel_transients = 2
        x = self.__check_wrap(x, vel_transients)
        v = (x[2:] - x[0:-2]) / (2*dt)
        return np.squeeze(v)
        
    def acceleration(self, x, dt=1):
        acc_transients = 2
        x = self.__check_wrap(x, acc_transients)
        a = (x[2:] - 2.*x[1:-1] + x[0:-2]) / (dt**2)
        return np.squeeze(a)
    
    def jerk(self, x, dt=1):
        jrk_transients = 4
        x = self.__check_wrap(x, jrk_transients)        
        j = (x[4:] - 2.*x[3:-1] + 2.*x[1:-3] - x[0:-4]) / (2*dt**3)
        return np.squeeze(j)
    
    def __curv2d(self, vel, acc):
        A = np.stack((acc, vel), axis=-1)
        vel_dot = np.sum(np.multiply(vel, vel), axis=-1)
        return np.abs(np.linalg.det(A)) / np.power(vel_dot, 3.0/2.0)
        
    def __curv3d(self, vel, acc):
        norm_cross = self.norm(np.cross(acc, vel))
        vel_cubed = np.power(self.norm(vel), 3.0)
        return norm_cross / (vel_cubed + self.eps)
    
    def curvature(self, vel, acc):
        dim = vel.shape[-1]
        if not (1 < dim < 4):
            s = inspect.stack()[0].function
            raise InputError(
                'An error occured while calling function \"%s\"'% s,
                'Curvature is udefined for signals with dimensions < 2 or > 3'
            )
        
        if dim == 2:
            return self.__curv2d(vel, acc)
        elif dim == 3:
            return self.__curv3d(vel, acc)
        
        
    def convex_hull(self, x):
        hulls = []
        for i in range(x.shape[0]):
            hull = ConvexHull(x[i])
            hulls.append(hull)
        
        return hulls
        
    def bounding_box(self, x):
        rect = np.zeros((x.shape[0], x.shape[-1], 2))
        for i in range(x.shape[-1]):
            dim = x[:, :, i]
            rect[:, i, 0] = np.amin(dim, axis=1)
            rect[:, i, 1] = np.amax(dim, axis=1)
        
        return rect
        
    def bounding_sphere(self, x):
        com = self.center_of_mass(x)
        cen = com.reshape((-1, 1, com.shape[-1]))
        radii = self.norm(x - cen)
        radius = np.amax(radii, axis=1)
        return (cen, radius)
        

    '''
        UTILITY METHODS
    '''
    
    def get_head_dir(self, x, hier):
        headidx = hier["head"]
        lhumidx = hier["lhumerus"]
        rhumidx = hier["rhumerus"]
        head = x[:, headidx]
        lhum = x[:, lhumidx]
        rhum = x[:, rhumidx]
        
        h2l = lhum - head
        h2r = rhum - head
        return np.cross(h2l, h2r)
        
    def get_root_dir(self, x, hier, dt=1):
        rootidx = hier["root"]
        root = x[:, rootidx]
        return self.velocity(root, dt)
    
    def set_wrap_func(self, mode):
        self.wrap_func = self.__get_wrap_func(mode=mode)

    def __check_wrap(self, x, transients):
        if self.wrap_func:
            x = self.wrap_func(x, transients)   
        return x
        
    def __get_wrap_func(self, mode='repeat'):
        def wrap_edges(vec, k):
            d = vec.shape
            if len(d) == 1:
                d += (1, )

            tmp = np.resize(vec, (d[0] + k, *d[1:]))
            tmp[k//2:] = tmp[:-k//2]

            if mode == 'repeat':
                tmp[:k//2] = vec[0]
                tmp[-k//2:] = vec[-1]

            elif mode == 'zero':
                tmp[:k//2] = tmp[-k//2:] = 0

            elif mode == 'wrap':
                tmp[:k//2] = vec[-k//2:]
                tmp[-k//2:] = vec[:k//2]

            return tmp
        
        return wrap_edges if mode else None 

# Mid-Level Motion Descriptors
The `LabanDescriptor` class below implements methods for computing the Laban efforts *weight, space, time* and *flow*. Formulas are derived from *Larboulette & Gibet (2016, referenced earlier)*. This class represent one possible abstraction of the third level of the earlier described framework. 

In [6]:
class LabanDescriptor(MotionDescriptor):
    
    def __init__(self, motion_data, sr=1, hier=None):
        super().__init__()        
        self.euler_positions = motion_data
        
        self.fs = 1 / sr
        self.markerids   = None
        self.num_markers = motion_data.shape[1]
        self.std_weights = np.ones(self.num_markers)
        
        if hier:
            self.markerids = {
                hier[k][b"name"].decode('utf-8'): k \
                    for k in hier.keys()
            }
        
            
    def weight(self, time_window_len, weights=[]):
        vel = self.norm(self.velocity(self.euler_positions, self.fs))
        vel_windowed = self.__window_input(vel, time_window_len)
        _weights = self.__check_weights(weights)
        weight_k = np.sum(np.multiply(np.power(vel_windowed, 2), _weights), axis=-1)
        
        return np.amax(weight_k, axis=-1)
    
    def space(self, time_window_len, weights=[]):
        pos = self.euler_positions
        pos_windowed = self.__window_input(pos, time_window_len)
        
        displacement_n = pos_windowed[:, 1:] - pos_windowed[:, 0:-1]
        displacement_t = pos_windowed[:, -1] - pos_windowed[:, 0]
        
        disp_n_norm = self.norm(displacement_n)
        disp_t_norm = self.norm(displacement_t)
        
        disp_n_sum = np.sum(disp_n_norm, axis=1)
        
        #disp_avg = disp_n_sum / (disp_t_norm + self.eps)
        disp_avg = disp_n_sum / disp_t_norm
        
        _weights = self.__check_weights(weights)
        disp_avg_s = np.multiply(disp_avg, _weights)
        
        return np.sum(disp_avg_s, axis=-1)
                
    def time(self, time_window_len, weights=[]):
        acc = self.norm(self.acceleration(self.euler_positions, self.fs))
        acc_windowed = self.__window_input(acc, time_window_len)
        time_k = np.sum(acc_windowed, axis=1) / time_window_len
        _weights = self.__check_weights(weights)
        return np.sum(np.multiply(time_k, _weights), axis=-1)
        
    def flow(self, time_window_len, weights=[]):
        jrk = self.norm(self.jerk(self.euler_positions, self.fs))
        jrk_windowed = self.__window_input(jrk, time_window_len)
        flow_k = np.sum(jrk_windowed, axis=1) / time_window_len
        _weights = self.__check_weights(weights)
        return np.sum(np.multiply(flow_k, _weights), axis=-1)
    
    def space_inner(self, time_window_len):        
        if not self.markerids:
            print('No markers associated with this instance')
            return
        
        head_normal = self.get_head_dir(
            self.euler_positions,
            self.markerids
        )
        
        root_vel = self.get_root_dir(
            self.euler_positions,
            self.markerids,
            self.fs
        )
        
        xi = np.multiply(head_normal[:, 0], root_vel[:, 0])
        yi = np.multiply(head_normal[:, 1], root_vel[:, 1])
        zi = np.multiply(head_normal[:, 2], root_vel[:, 2])
        inner = xi + yi + zi
        
        win = self.__window_input(inner, time_window_len)
        return np.squeeze(np.mean(win, axis=1))
        
        
    '''
        UTILITY METHODS
    '''
        
    def get_weight_vector_for(self, **kwargs):
        if not self.markerids:
            print('No markers associated with this instance')
            return
        
        weights = [0 for _ in self.markerids.keys()]
        for k, v in kwargs.items():
            if not k in self.markerids:
                s = inspect.stack()[1].function
                raise InputError(
                    'An error occured while calling function \"%s\"'% s,
                    'Argument \"%s\" is not valid for this function'% k
                )
                
            markeridx = self.markerids[k]
            weights[markeridx] = v
        
        return weights
                
    
    def __check_weights(self, weights):
        if not np.any(weights):
            return self.std_weights
        
        if len(weights) is not self.num_markers:
            s = inspect.stack()[1].function
            raise InputError(
                'An error occured while calling function \"%s\"'% s,
                'Input number of weights does not match number of markers'
            )
            
        return weights
    
    def __window_input(self, ndsig, wlen):
        d = ndsig.shape
        if len(d) == 1:
            d += (1, )
        
        return np.resize(ndsig, (d[0]//wlen, wlen, *d[1:]))

# High-Level Motion (Expressive Qualities)
The final (i.e. fourth) level of Camurri et al. (2013)'s framework is current very simple and consist of just a single class, the `MotionExpression` class. Presently, the class reveals a method that can produce a list of time-indeces where there is a significant peak in one the Laban efforts. The assumption is that these peaks reveal which sub-dimension of the Laban efforts (e.g. *weight*, hence light or strong) the motion within that time-frame could fall within. This approach is inspired by the work in *Hachimura et al. (2005)*. 

#### Reference:
<sup>K. Hachimura, K. Takashina and M. Yoshimura, "Analysis and evaluation of dancing movement based on LMA," *ROMAN 2005. IEEE International Workshop on Robot and Human Interactive Communication, 2005., Nashville, TN, USA, 2005*, pp. 294-299, doi: 10.1109/ROMAN.2005.1513794.</sup>

In [7]:
class MotionExpression(object):
    
    def __init__(self):
        
        self.tmeans = {
            'weight': 1.0729,
            'space': -0.4684,
            'time': 4.6185
        }
        
        self.tfunc = {
            'weight': self.__log_std_norm,
            'space': self.__std_norm,
            'time': self.__log_std_norm 
        }
        
    def get_sigpeaks(self, effort, effortdata, sigma=1):
        if not effort in self.tmeans:
            print('no threshold available for effort %s'% effort)
            return
        
        tfunc  = self.tfunc[effort]
        thresh = self.tmeans[effort]
        
        effortdata = tfunc(effortdata)
        tindx_lo = np.argwhere(effortdata <= (thresh - sigma))
        tindx_hi = np.argwhere(effortdata >  (thresh + sigma))
        
        return (tindx_lo, tindx_hi)
    
    
    def sort_idx_diff(self, indc, maxdiff=2):
        if not np.any(indc):
            return np.array([])
        
        x = indc.flatten()
        diff = np.diff(x)
        inds = np.argwhere(diff > 2).flatten()

        groups = []
        inds = [*inds, len(x)]
        beg, end = 0, inds[0] + 1
        groups.append(x[beg:end])
        for i in range(1, len(inds)):
            beg = end
            end = inds[i] + 1
            groups.append(x[beg:end])      
                
        ranges = []
        for g in groups:
            mi, ma = g[0], g[-1]
            ranges.append((mi, ma))
        
        return ranges
        
    def __std_norm(self, data):
        return data / np.std(data)
    
    def __log_std_norm(self, data):
        datalog = np.log10(data)
        return self.__std_norm(datalog)

# Plotting Functionality
The class below is simply a container for functions to plot various graphics.

In [8]:
class MotionGraphics(object):
    
    RENDERS = "notebook+plotly_mimetype+pdf"
    
    def __init__(self):
        pass
    
    def __color_grad(self, c1, c2, mix=0.0):
        c1=np.array(mpl.colors.to_rgb(c1))
        c2=np.array(mpl.colors.to_rgb(c2))
        return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
    
    def __skeleton_trace(self, pose, hier, colors):
        traces = []
        for key, color in zip(hier.keys(), colors):
            child_key = key
            child = child_key
            if hier[child_key][b"parent"] is not None:
                parent = hier[child_key][b"parent"]
                xs = [pose[child, 0], pose[parent, 0]]
                ys = [pose[child, 1], pose[parent, 1]]
                zs = [pose[child, 2], pose[parent, 2]]

                traces.append(go.Scatter3d(
                    x=zs, y=xs, z=ys,
                    marker=dict(
                        size=3,
                        color=color
                    ),
                    line=dict(
                        color='darkblue',
                        width=2
                    ),
                    name=hier[key][b"name"].decode("utf-8")
                ))
                
        return traces
    
    def __vector_trace(self, cinput, args):
        
        orego = cinput[0]
        v = cinput[1]
        scale = args[0]
        
        v = v / np.linalg.norm(v, axis=-1)
        endp = orego + v*scale
        
        vtrace = go.Scatter3d(
            x = [orego[2], endp[2]],
            y = [orego[0], endp[0]],
            z = [orego[1], endp[1]],
            marker = dict(
                size = 2,
                color = "rgb(84,48,5)"
            ),
            line = dict(
                color = "rgb(84,48,5)",
                width = 6
            )
        )
        
        return vtrace
    
    def __skeleton_limits(self, poses):
        lims = []
        for d in range(poses.shape[-1]):
            c = poses[:, :, d]
            lmin = np.amin(c)
            lmax = np.amax(c)
            lims.append((lmin, lmax))
            
        return lims
    
    def line_plot(self, data, xlabel='Placeholder', ylabel='Placeholder', w=600, h=500):
        data = data.flatten()
        miny = np.amin(data) - 1
        maxy = np.amax(data) + 1
        
        fig = go.Figure(
            layout=dict(
                width=w,
                height=h,
                yaxis_range=[miny, maxy],
                xaxis_title=xlabel,
                yaxis_title=ylabel
            )
        )
        
        xs = np.arange(len(data))
        fig.add_trace(
            go.Scatter(
                x=xs,
                y=data,
                mode='lines'
            )
        )
        
        return fig, (miny, maxy)
        
    
    def shaded_plot(self, data, bounds, xlabel='Placeholder', ylabel='Placeholder', w=600, h=500):
        fig, (miny, maxy) = self.line_plot(data, xlabel, ylabel, w, h)
        
        if not np.any(bounds):
            return fig
        
        for r in bounds:
            mi = r[0]
            ma = r[1]
            xs = [mi, mi, ma, ma]
            ys = [miny, maxy, maxy, miny]

            t = go.Scatter(
                x=xs,
                y=ys,
                fill='toself',
                fillcolor='blue',
                line_color='blue',
                mode = 'lines',
                opacity=0.3
            )

            fig.add_trace(t)
            
        return fig
        
    
    def draw_skeleton(self, pose, hier, title='Skeleton', w=600, h=600, cmap=('red', 'orange')):
        fig = make_subplots()
        nmarks = pose.shape[0]
        colors = [
            self.__color_grad(cmap[0], cmap[1], k/nmarks) \
                for k in range(nmarks)
        ]
        
        traces = self.__skeleton_trace(pose, hier, colors)
        fig.add_traces(traces)
        
        l = self.__skeleton_limits(
            pose.reshape((1, pose.shape[0], pose.shape[1]))
        )
                
        fig.update_layout(
            title=title,
            width=w, height=h,
            scene=dict(
                xaxis=dict(range=[l[2][0] - .5, l[2][1] + .5], autorange=False),
                yaxis=dict(range=[l[0][0] -  1, l[0][1] +  1], autorange=False),
                zaxis=dict(range=[l[1][0] - .5, l[1][1] + .5], autorange=False),
            )
        )
        
        fig.show(renderer=MotionGraphics.RENDERS)
    
    def __fargs(self, duration):
        return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "elastic"}
        }
    
    def __setup_sliders(self, frames):
        return [{
            "pad": {"b": 10, "t": 60},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": [{
                    "args": [[f.name], self.__fargs(0)],
                    "label": str(k),
                    "method": "animate",
                } for k, f in enumerate(frames)
            ],
        }]
    
    def animate_skeleton(self, poses, hier, beg, end, step=1, w=600, h=600, fr=30, cmap=('red', 'orange')):
        nmarks = poses.shape[1]
        colors = [
            self.__color_grad(cmap[0], cmap[1], k/nmarks) \
                for k in range(nmarks)
        ]
        
        data = []
        for k in range(beg, end, step):
            pose = poses[k]
            traces = self.__skeleton_trace(pose, hier, colors)            
            data.append(traces)
            
        frames = [
            go.Frame(data=data[k], name=str(k))\
                for k in range(len(data))
        ]
        
        fig = go.Figure(frames=frames)
        fig.add_traces(data[0])

        sliders = self.__setup_sliders(fig.frames)
        l = self.__skeleton_limits(poses[beg:end:step])
        
        fig.update_layout(
            title='Motion Clip',
            width=w,
            height=h,
            scene=dict(
                xaxis=dict(range=[l[2][0] - .5, l[2][1] + .5], autorange=False),
                yaxis=dict(range=[l[0][0] -  1, l[0][1] +  1], autorange=False),
                zaxis=dict(range=[l[1][0] - .5, l[1][1] + .5], autorange=False),
                aspectratio=dict(x=1, y=1, z=1)
            ),
            updatemenus = [
                {
                    "buttons": [
                        {
                            "args": [None, self.__fargs(fr)],
                            "label": "&#9654;", 
                            "method": "animate",
                        },
                        {
                            "args": [[None], self.__fargs(0)],
                            "label": "&#9724;",
                            "method": "animate",
                        },
                    ],
                    "direction": "left",
                    "pad": {"r": 10, "t": 70},
                    "type": "buttons",
                    "x": 0.1,
                    "y": 0,
                }
            ],
            sliders=sliders
        )
        return fig        

# [Example] Usage of the Framework

In [15]:
# Initialize classes
mgraph = MotionGraphics()
mextra = MotionExtractor()

# Extract an example motion
mextra.set_parse_params(
    path=None,
    subject_number=135,
    trial_id = 'Trial4'
)

data, hier, sr = mextra.parse_motion()
print(data.shape, sr)

# Filter the data (with an IIR filter with normalized cutoff 0.2)
data_filt = mextra.filter_motion(data, '0.2')

# Setup and initilize motion descriptor class
manalysis = LabanDescriptor(data_filt, sr, hier=hier)

(1315, 31, 3) 120


### Extract Some "Mid-Level" Descriptors

In [32]:
# Weight Descriptor (Laban) is computed over the entire body
ww = []
weight = manalysis.weight(5, ww)

# Time Descriptor (Laban) is computed over the entire body
tw = []
time = manalysis.time(20, tw)

# Space Descriptor (Laban) is computed over the entire body
space = manalysis.space_inner(20)

# Flow Descriptor (Laban) is computed over the entire body
fw = [] 
flow = manalysis.flow(20, fw)

In [34]:
motionqual = MotionExpression()

effort = 'weight'
effortdata = weight

lo, hi = motionqual.get_sigpeaks(
    effort=effort, 
    effortdata=effortdata,
    sigma=2
)

bounds = motionqual.sort_idx_diff(hi)
fig = mgraph.shaded_plot(
    effortdata, 
    bounds, 
    xlabel='Time windows', 
    ylabel=effort
)

fig.show()
print(bounds)

[]


In [30]:
# Plot an animation
region = bounds[1]

mm_clip = lambda x, l, u: max(l, min(u, x))

minv, maxv = 0, data_filt.shape[0]

beg  = mm_clip(region[0]*20 - 40, minv, maxv)
end  = mm_clip(region[1]*20 + 40, minv, maxv)
step = int(sr/30) 

fig = mgraph.animate_skeleton(
    data_filt, 
    hier, 
    beg, end, 
    step=step,
    fr=30
)

py.iplot(fig)

# [Example 2] Read and Save Several Motions

In [ ]:
raise Error()

subs = [3, 35, 56, 62, 120, 126]
ntri = [4, 20, 8, 25,  20,  14]

hier = None
for s, t in zip(subs, ntri):
    for trial in range(t):
        me = MotionExtractor()
        me.set_parse_params(
            path=None,
            subject_number=s,
            trial_id = 'Trial%s'%(trial+1)
        )
        
        data, hier, sr = me.parse_motion()
        np.save('./MoCapNPY/mSub{0}Trial{1}'.format(s, trial+1), data)
        del me

In [ ]:
import glob
me = MotionExtractor()

labw = []
labs = []
labt = []
labf = []

for file in glob.glob('./MoCapNPY/*.npy'):
    data = np.load(file)
    data = me.filter_motion(data, '0.3')
    ld = LabanDescriptor(data, sr, hier=hier)
    
    w = ld.weight(20)
    labw.append(w)
    
    s = ld.space_inner(20)
    labs.append(s)
    
    t = ld.time(20)
    labt.append(t)
    
    f = ld.flow(20)
    labf.append(t)
    

npw = np.concatenate(labw, axis=0)
nps = np.concatenate(labs, axis=0)
npt = np.concatenate(labt, axis=0)
npf = np.concatenate(labf, axis=0)